<a href="https://colab.research.google.com/github/javmencia/COBWEBfiles/blob/main/SLErandomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (classification_report, confusion_matrix,
                            balanced_accuracy_score, cohen_kappa_score,
                            precision_score, recall_score, accuracy_score)
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
import random

# Set seeds for reproducibility
random.seed(1927)
np.random.seed(1927)

# --- Data Loading & Preprocessing ---
data = pd.read_csv('sledatacut.csv', parse_dates=['ASSDT'], low_memory=False)
dataorig = pd.read_csv('sledatacut.csv', parse_dates=['ASSDT'], low_memory=False)
data = data.sort_values(['PTNO', 'ASSDT'])

# Calculate time differences
data['time_since_first'] = data.groupby('PTNO')['ASSDT'].transform(
    lambda x: (x - x.min()).dt.days
)
data['time_since_last'] = data.groupby('PTNO')['ASSDT'].transform(
    lambda x: x.diff().dt.days.fillna(0)
)

# Convert categorical EMPf to numerical
emp_mapping = {v: k for k, v in enumerate(data['EMPf'].unique())}
data['EMP_numeric'] = data['EMPf'].map(emp_mapping)

# Encode the target (end_state)
label_encoder = LabelEncoder()
data['end_state_encoded'] = label_encoder.fit_transform(data['end_state'])

# Encode steroid categories (Low/Medium/High)
steroid_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
data['STEROID_CAT_numeric'] = data['STEROID_CAT'].map(steroid_mapping)

# Features to use - including SDI (score_n) along with flares and steroid categories
features = ['severe_flare', 'mild_flare', 'total_flares',
            'STERDOSE', 'INCEPT', 'AMDOSE',
            'age_at_record', 'time_since_last', 'time_since_first',
            'visit_num', 'score_n', "AMS",  "SDI_Ocular", "SDI_Neuropsychiatric", "SDI_Renal",
            "SDI_Pulmonary", "SDI_Cardiovascular", "SDI_PeripheralVascular", "SDI_Gastrointestinal",
            "SDI_Musculoskeletal", "SDI_Dermatologic","SDI_Endocrine", "SDI_Malignancy", "SLEDAI_Constitutional",
            "SLEDAI_Cutaneous", "SLEDAI_Musculoskeletal", "SLEDAI_Serositis", "SLEDAI_Renal", "SLEDAI_Neurological",
            "SLEDAI_Hematological", "SLEDAI_Immunological", "SLEDAI_Vascular", "SLEDAI_Other"
            ]

# Add interaction terms between important features
data['sdi_flare_interaction'] = data['score_n'] * data['total_flares']

# Add temporal features
data['flares_per_month'] = data['total_flares'] / (data['time_since_first']/30 + 1)  # +1 to avoid division by zero

# Correct SDI change rate calculation - using time_since_last instead of index
data['sdi_change_rate'] = data.groupby('PTNO', group_keys=False).apply(
    lambda x: x['score_n'].diff().fillna(0) / (x['time_since_last']/30 + 1e-6)  # Small constant to avoid division by zero
)

# Update features list
features += ['sdi_flare_interaction', 'sdi_change_rate']

# Identify categorical columns (non-numeric)
categorical_cols = data[features].select_dtypes(include=['object', 'category']).columns

# Label encode categorical columns
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))

# Fill NA values - for flares and SDI we'll assume 0 if missing
flare_cols = ['severe_flares', 'mild_flares']
data[flare_cols] = data[flare_cols].fillna(0)
data['score_n'] = data['score_n'].fillna(0)  # SDI
data[features] = data[features].fillna(0)

# Normalize features
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])

# Create a custom aggregation dictionary
agg_dict = {feature: 'max' for feature in features}

# For temporal features, use last observation instead of max
temporal_features = ['time_since_first', 'time_since_last', 'visit_num']
for feature in temporal_features:
    if feature in agg_dict:
        agg_dict[feature] = 'last'

# Aggregate features
max_observations = data.groupby('PTNO').agg(agg_dict)

# Get the last observation for the target variable
last_target = data.groupby('PTNO')['end_state_encoded'].last()

X = max_observations.values
y = last_target.values

# --- Initialize Cross-Validation ---
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

cv_results = {
    'rf_val_accuracy': [],
    'rf_val_balanced_accuracy': [],
    'rf_val_precision_per_class': [],
    'rf_val_recall_per_class': [],
    'rf_models': []
}

# --- Cross-Validation Loop ---
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n=== Fold {fold + 1}/{n_splits} ===")

    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Class weights
    classes = np.unique(y_train)
    class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
    class_weight_dict = dict(enumerate(class_weights))

    # --- Random Forest Training ---
    rf = BalancedRandomForestClassifier(
        n_estimators=300,
        sampling_strategy='all',
        replacement=True,
        random_state=42,
        class_weight='balanced',
        max_depth=10,
        min_samples_leaf=5
    )
    rf.fit(X_train, y_train)

    # RF Validation Predictions
    y_val_pred_rf = rf.predict(X_val)
    y_val_pred_rf_proba = rf.predict_proba(X_val)

    # Store RF metrics
    cv_results['rf_val_accuracy'].append(accuracy_score(y_val, y_val_pred_rf))
    cv_results['rf_val_balanced_accuracy'].append(balanced_accuracy_score(y_val, y_val_pred_rf))
    cv_results['rf_val_precision_per_class'].append(precision_score(y_val, y_val_pred_rf, average=None))
    cv_results['rf_val_recall_per_class'].append(recall_score(y_val, y_val_pred_rf, average=None))
    cv_results['rf_models'].append(rf)

    # --- Fold Summary ---
    print(f"RF Val Accuracy: {cv_results['rf_val_accuracy'][-1]:.4f}")
    print(f"RF Val Balanced Accuracy: {cv_results['rf_val_balanced_accuracy'][-1]:.4f}")

# --- Final Cross-Validation Report ---
print("\n=== Final Cross-Validation Results ===")

# RF Performance
print("\nRF Metrics:")
print(f"Mean Accuracy: {np.mean(cv_results['rf_val_accuracy']):.4f} (±{np.std(cv_results['rf_val_accuracy']):.4f})")
print(f"Mean Balanced Accuracy: {np.mean(cv_results['rf_val_balanced_accuracy']):.4f} (±{np.std(cv_results['rf_val_balanced_accuracy']):.4f})")

# Per-class metrics (averaged across folds)
print("\nRF Per-Class Metrics (Averaged):")
avg_precision = np.mean(cv_results['rf_val_precision_per_class'], axis=0)
avg_recall = np.mean(cv_results['rf_val_recall_per_class'], axis=0)
for i, (prec, rec) in enumerate(zip(avg_precision, avg_recall)):
    print(f"Class {label_encoder.classes_[i]} - Precision: {prec:.4f}, Recall: {rec:.4f}")

# Feature Importance (from last RF fold)
print("\nTop 10 RF Features (from last fold):")
importances = pd.DataFrame({
    'Feature': features,
    'Importance': cv_results['rf_models'][-1].feature_importances_
}).sort_values('Importance', ascending=False)
print(importances.head(10))

/tmp/ipython-input-4-414565143.py:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data['sdi_change_rate'] = data.groupby('PTNO', group_keys=False).apply(



=== Fold 1/5 ===


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RF Val Accuracy: 0.0585
RF Val Balanced Accuracy: 0.3384

=== Fold 2/5 ===


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RF Val Accuracy: 0.0634
RF Val Balanced Accuracy: 0.3709

=== Fold 4/5 ===


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RF Val Accuracy: 0.0683
RF Val Balanced Accuracy: 0.3714

=== Fold 5/5 ===
RF Val Accuracy: 0.0634
RF Val Balanced Accuracy: 0.3714

=== Final Cross-Validation Results ===

RF Metrics:
Mean Accuracy: 0.0654 (±0.0050)
Mean Balanced Accuracy: 0.3698 (±0.0186)

RF Per-Class Metrics (Averaged):
Class Disability Benefit - Precision: 0.0000, Recall: 0.0000
Class Early Retirement - Precision: 0.1459, Recall: 0.7133
Class Economically Inactive - Precision: 0.0903, Recall: 0.5143
Class Employed - Precision: 0.0000, Recall: 0.0000
Class Employed (Died) - Precision: 0.0375, Recall: 0.8133
Class Unemployed - Precision: 0.0684, Recall: 0.1778

Top 10 RF Features (from last fold):
                  Feature  Importance
6           age_at_record    0.181443
8        time_since_first    0.161861
9               visit_num    0.105316
7         time_since_last    0.074712
11                    AMS    0.069115
3                STERDOSE    0.059887
5                  AMDOSE    0.043923
28    SLEDAI_Neurolo

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (classification_report, confusion_matrix,
                            balanced_accuracy_score, cohen_kappa_score,
                            precision_score, recall_score, accuracy_score)
from xgboost import XGBClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
import random

# Set seeds for reproducibility
random.seed(1927)
np.random.seed(1927)

# --- Data Loading & Preprocessing ---
data = pd.read_csv('sledatacut.csv', parse_dates=['ASSDT'], low_memory=False)
dataorig = pd.read_csv('sledatacut.csv', parse_dates=['ASSDT'], low_memory=False)
data = data.sort_values(['PTNO', 'ASSDT'])

# Calculate time differences
data['time_since_first'] = data.groupby('PTNO')['ASSDT'].transform(
    lambda x: (x - x.min()).dt.days
)
data['time_since_last'] = data.groupby('PTNO')['ASSDT'].transform(
    lambda x: x.diff().dt.days.fillna(0)
)

# Convert categorical EMPf to numerical
emp_mapping = {v: k for k, v in enumerate(data['EMPf'].unique())}
data['EMP_numeric'] = data['EMPf'].map(emp_mapping)

# Encode the target (end_state)
label_encoder = LabelEncoder()
data['end_state_encoded'] = label_encoder.fit_transform(data['end_state'])

# Encode steroid categories (Low/Medium/High)
steroid_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
data['STEROID_CAT_numeric'] = data['STEROID_CAT'].map(steroid_mapping)

# Features to use - including SDI (score_n) along with flares and steroid categories
features = ['severe_flare', 'mild_flare', 'total_flares',
            'STERDOSE', 'INCEPT', 'AMDOSE',
            'age_at_record', 'time_since_last', 'time_since_first',
            'visit_num', 'score_n', "AMS",  "SDI_Ocular", "SDI_Neuropsychiatric", "SDI_Renal",
            "SDI_Pulmonary", "SDI_Cardiovascular", "SDI_PeripheralVascular", "SDI_Gastrointestinal",
            "SDI_Musculoskeletal", "SDI_Dermatologic","SDI_Endocrine", "SDI_Malignancy", "SLEDAI_Constitutional",
            "SLEDAI_Cutaneous", "SLEDAI_Musculoskeletal", "SLEDAI_Serositis", "SLEDAI_Renal", "SLEDAI_Neurological",
            "SLEDAI_Hematological", "SLEDAI_Immunological", "SLEDAI_Vascular", "SLEDAI_Other"
            ]

# Add interaction terms between important features
data['sdi_flare_interaction'] = data['score_n'] * data['total_flares']

# Add temporal features
data['flares_per_month'] = data['total_flares'] / (data['time_since_first']/30 + 1)  # +1 to avoid division by zero

# Correct SDI change rate calculation - using time_since_last instead of index
data['sdi_change_rate'] = data.groupby('PTNO', group_keys=False).apply(
    lambda x: x['score_n'].diff().fillna(0) / (x['time_since_last']/30 + 1e-6)  # Small constant to avoid division by zero
)

# Update features list
features += ['sdi_flare_interaction', 'sdi_change_rate']

# Identify categorical columns (non-numeric)
categorical_cols = data[features].select_dtypes(include=['object', 'category']).columns

# Label encode categorical columns
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))

# Fill NA values - for flares and SDI we'll assume 0 if missing
flare_cols = ['severe_flares', 'mild_flares']
data[flare_cols] = data[flare_cols].fillna(0)
data['score_n'] = data['score_n'].fillna(0)  # SDI
data[features] = data[features].fillna(0)

# Normalize features
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])

# Create a custom aggregation dictionary
agg_dict = {feature: 'max' for feature in features}

# For temporal features, use last observation instead of max
temporal_features = ['time_since_first', 'time_since_last', 'visit_num']
for feature in temporal_features:
    if feature in agg_dict:
        agg_dict[feature] = 'last'

# Aggregate features
max_observations = data.groupby('PTNO').agg(agg_dict)

# Get the last observation for the target variable
last_target = data.groupby('PTNO')['end_state_encoded'].last()

X = max_observations.values
y = last_target.values

# --- Initialize Cross-Validation ---
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

cv_results = {
    'xgb_val_accuracy': [],
    'xgb_val_balanced_accuracy': [],
    'xgb_val_precision_per_class': [],
    'xgb_val_recall_per_class': [],
    'xgb_models': []
}

# --- Cross-Validation Loop ---
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n=== Fold {fold + 1}/{n_splits} ===")

    # Split data
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Class weights
    classes = np.unique(y_train)
    class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
    class_weight_dict = dict(enumerate(class_weights))

    # --- XGBoost Training ---
    xgb = XGBClassifier(
        n_estimators=300,
        learning_rate=0.1,
        max_depth=6,
        min_child_weight=3,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='multi:softprob',
        num_class=len(np.unique(y)),
        random_state=42,
        scale_pos_weight=len(y_train[y_train==0])/len(y_train[y_train==1]) if len(np.unique(y)) == 2 else None,
        early_stopping_rounds=20,
        eval_metric='mlogloss'
    )

    xgb.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # XGBoost Validation Predictions
    y_val_pred_xgb = xgb.predict(X_val)
    y_val_pred_xgb_proba = xgb.predict_proba(X_val)

    # Store XGBoost metrics
    cv_results['xgb_val_accuracy'].append(accuracy_score(y_val, y_val_pred_xgb))
    cv_results['xgb_val_balanced_accuracy'].append(balanced_accuracy_score(y_val, y_val_pred_xgb))
    cv_results['xgb_val_precision_per_class'].append(precision_score(y_val, y_val_pred_xgb, average=None))
    cv_results['xgb_val_recall_per_class'].append(recall_score(y_val, y_val_pred_xgb, average=None))
    cv_results['xgb_models'].append(xgb)

    # --- Fold Summary ---
    print(f"XGB Val Accuracy: {cv_results['xgb_val_accuracy'][-1]:.4f}")
    print(f"XGB Val Balanced Accuracy: {cv_results['xgb_val_balanced_accuracy'][-1]:.4f}")

# --- Final Cross-Validation Report ---
print("\n=== Final Cross-Validation Results ===")

# XGBoost Performance
print("\nXGBoost Metrics:")
print(f"Mean Accuracy: {np.mean(cv_results['xgb_val_accuracy']):.4f} (±{np.std(cv_results['xgb_val_accuracy']):.4f})")
print(f"Mean Balanced Accuracy: {np.mean(cv_results['xgb_val_balanced_accuracy']):.4f} (±{np.std(cv_results['xgb_val_balanced_accuracy']):.4f})")

# Per-class metrics (averaged across folds)
print("\nXGBoost Per-Class Metrics (Averaged):")
avg_precision = np.mean(cv_results['xgb_val_precision_per_class'], axis=0)
avg_recall = np.mean(cv_results['xgb_val_recall_per_class'], axis=0)
for i, (prec, rec) in enumerate(zip(avg_precision, avg_recall)):
    print(f"Class {label_encoder.classes_[i]} - Precision: {prec:.4f}, Recall: {rec:.4f}")

# Feature Importance (from last XGBoost fold)
print("\nTop 10 XGBoost Features (from last fold):")
importances = pd.DataFrame({
    'Feature': features,
    'Importance': cv_results['xgb_models'][-1].feature_importances_
}).sort_values('Importance', ascending=False)
print(importances.head(10))

/tmp/ipython-input-5-1875949777.py:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data['sdi_change_rate'] = data.groupby('PTNO', group_keys=False).apply(



=== Fold 1/5 ===


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


XGB Val Accuracy: 0.7854
XGB Val Balanced Accuracy: 0.2776

=== Fold 2/5 ===


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


XGB Val Accuracy: 0.8049
XGB Val Balanced Accuracy: 0.3493

=== Fold 4/5 ===


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


XGB Val Accuracy: 0.7951
XGB Val Balanced Accuracy: 0.3484

=== Fold 5/5 ===
XGB Val Accuracy: 0.8146
XGB Val Balanced Accuracy: 0.3004

=== Final Cross-Validation Results ===

XGBoost Metrics:
Mean Accuracy: 0.8010 (±0.0099)
Mean Balanced Accuracy: 0.3111 (±0.0319)

XGBoost Per-Class Metrics (Averaged):
Class Disability Benefit - Precision: 0.5536, Recall: 0.5508
Class Early Retirement - Precision: 0.4333, Recall: 0.1933
Class Economically Inactive - Precision: 0.2000, Recall: 0.0857
Class Employed - Precision: 0.8600, Recall: 0.9699
Class Employed (Died) - Precision: 0.0000, Recall: 0.0000
Class Unemployed - Precision: 0.1400, Recall: 0.0667

Top 10 XGBoost Features (from last fold):
                Feature  Importance
8      time_since_first    0.069420
9             visit_num    0.056362
27         SLEDAI_Renal    0.046867
3              STERDOSE    0.046167
6         age_at_record    0.044019
0          severe_flare    0.043402
28  SLEDAI_Neurological    0.038246
34      sdi_chang

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (classification_report, confusion_matrix,
                            balanced_accuracy_score, cohen_kappa_score,
                            precision_score, recall_score, accuracy_score)
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier, EasyEnsembleClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from concurrent.futures import ThreadPoolExecutor
import random

# Set seeds for reproducibility
random.seed(1927)
np.random.seed(1927)

# --- Data Loading & Preprocessing ---
data = pd.read_csv('sledatacut.csv', parse_dates=['ASSDT'], low_memory=False)
data = data.sort_values(['PTNO', 'ASSDT'])

# Time difference features
data['time_since_first'] = data.groupby('PTNO')['ASSDT'].transform(
    lambda x: (x - x.min()).dt.days
)
data['time_since_last'] = data.groupby('PTNO')['ASSDT'].transform(
    lambda x: x.diff().dt.days.fillna(0)
)

# Feature engineering
data['flares_per_month'] = data['total_flares'] / (data['time_since_first']/30 + 1)
data['steroid_flare_interaction'] = data['STERDOSE'] * data['total_flares']
data['sdi_flare_interaction'] = data['score_n'] * data['total_flares']

# Handle rare classes
class_counts = data['end_state'].value_counts()
rare_classes = class_counts[class_counts < 10].index
data['end_state'] = data['end_state'].replace(rare_classes, 'Other')

label_encoder = LabelEncoder()
data['end_state_encoded'] = label_encoder.fit_transform(data['end_state'])

# Features
features = [
    'severe_flare', 'mild_flare', 'total_flares',
    'STERDOSE', 'INCEPT', 'AMDOSE',
    'age_at_record', 'time_since_last', 'time_since_first',
    'visit_num', 'score_n', "AMS",
    'flares_per_month', 'steroid_flare_interaction', 'sdi_flare_interaction'
]

# Handle missing data
for col in features:
    if data[col].dtype in ['float64', 'int64']:
        data[col] = data[col].fillna(data[col].median())
    else:
        data[col] = data[col].fillna('missing')

# Scaling
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])

# Aggregate to patient-level
agg_dict = {feature: 'last' for feature in features}
agg_dict.update({
    'total_flares': 'sum',
    'severe_flare': 'sum',
    'mild_flare': 'sum',
    'score_n': 'max'
})

max_observations = data.groupby('PTNO').agg(agg_dict)
last_target = data.groupby('PTNO')['end_state_encoded'].last()

X = max_observations.values
y = last_target.values

# --- Model Definitions ---
def get_xgb_model():
    return Pipeline([
        ('smote', SMOTE(random_state=42)),
        ('classifier', XGBClassifier(
            n_estimators=300,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            scale_pos_weight=len(y[y==0])/len(y[y==1]) if len(np.unique(y)) == 2 else None,
            eval_metric='mlogloss',
            n_jobs=-1
        ))
    ])

def get_svm_model():
    return Pipeline([
        ('smote', SMOTE(random_state=42)),
        ('classifier', SVC(
            C=1.0,
            kernel='rbf',
            gamma='scale',
            class_weight='balanced',
            probability=True,
            random_state=42
        ))
    ])

def get_easy_ensemble_model():
    return EasyEnsembleClassifier(
        n_estimators=10,
        base_estimator=RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            class_weight='balanced',
            random_state=42
        ),
        random_state=42,
        n_jobs=-1
    )

def get_rf_model():
    return BalancedRandomForestClassifier(
        n_estimators=300,
        max_depth=10,
        min_samples_leaf=3,
        class_weight='balanced_subsample',
        random_state=42,
        n_jobs=-1
    )

# --- Parallel Cross-Validation ---
def run_cv(model_fn, model_name):
    print(f"\n=== Running {model_name} ===")
    n_splits = 5
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    results = {
        'accuracy': [],
        'balanced_accuracy': [],
        'precision': [],
        'recall': [],
        'f1': []
    }

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = model_fn()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)

        results['accuracy'].append(accuracy_score(y_val, y_pred))
        results['balanced_accuracy'].append(balanced_accuracy_score(y_val, y_pred))
        results['precision'].append(precision_score(y_val, y_pred, average='weighted', zero_division=0))
        results['recall'].append(recall_score(y_val, y_pred, average='weighted'))
        results['f1'].append(f1_score(y_val, y_pred, average='weighted'))

        print(f"{model_name} Fold {fold+1}: Balanced Acc = {results['balanced_accuracy'][-1]:.4f}")

    return {
        'model_name': model_name,
        'mean_accuracy': np.mean(results['accuracy']),
        'mean_balanced_accuracy': np.mean(results['balanced_accuracy']),
        'std_balanced_accuracy': np.std(results['balanced_accuracy']),
        'mean_precision': np.mean(results['precision']),
        'mean_recall': np.mean(results['recall']),
        'mean_f1': np.mean(results['f1'])
    }

# Run all models in parallel
models = [
    (get_xgb_model, "XGBoost"),
    (get_svm_model, "SVM"),
    (get_easy_ensemble_model, "EasyEnsemble"),
    (get_rf_model, "RandomForest")
]

with ThreadPoolExecutor() as executor:
    results = list(executor.map(lambda x: run_cv(x[0], x[1]), models))

# --- Results Summary ---
print("\n=== Final Model Comparison ===")
for res in results:
    print(f"\n{res['model_name']}:")
    print(f"Accuracy: {res['mean_accuracy']:.4f}")
    print(f"Balanced Accuracy: {res['mean_balanced_accuracy']:.4f} (±{res['std_balanced_accuracy']:.4f})")
    print(f"Precision: {res['mean_precision']:.4f}")
    print(f"Recall: {res['mean_recall']:.4f}")
    print(f"F1: {res['mean_f1']:.4f}")


=== Running XGBoost ===

=== Running SVM ===

=== Running EasyEnsemble ===

=== Running RandomForest ===
RandomForest Fold 1: Balanced Acc = 0.3677
RandomForest Fold 2: Balanced Acc = 0.4001
RandomForest Fold 3: Balanced Acc = 0.3949
XGBoost Fold 1: Balanced Acc = 0.3048
RandomForest Fold 4: Balanced Acc = 0.4282
RandomForest Fold 5: Balanced Acc = 0.3829
SVM Fold 1: Balanced Acc = 0.2925
XGBoost Fold 2: Balanced Acc = 0.3512
SVM Fold 2: Balanced Acc = 0.3638
XGBoost Fold 3: Balanced Acc = 0.4082
XGBoost Fold 4: Balanced Acc = 0.4104
SVM Fold 3: Balanced Acc = 0.3018
XGBoost Fold 5: Balanced Acc = 0.3681
SVM Fold 4: Balanced Acc = 0.3506
SVM Fold 5: Balanced Acc = 0.2481


TypeError: EasyEnsembleClassifier.__init__() got an unexpected keyword argument 'base_estimator'

In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score, make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import random
from scipy.stats import randint, uniform

# Set seeds for reproducibility
random.seed(1927)
np.random.seed(1927)

# --- Data Loading & Preprocessing ---
data = pd.read_csv('sledatacut.csv', parse_dates=['ASSDT'], low_memory=False)
data = data.sort_values(['PTNO', 'ASSDT'])

# --- Enhanced Feature Engineering ---
# Time-based features
data['time_since_first'] = data.groupby('PTNO')['ASSDT'].transform(
    lambda x: (x - x.min()).dt.days
)
data['time_since_last'] = data.groupby('PTNO')['ASSDT'].transform(
    lambda x: x.diff().dt.days.fillna(0)
)

# Interaction terms
data['steroid_flare_interaction'] = data['STERDOSE'] * data['total_flares']
data['sdi_age_interaction'] = data['score_n'] * data['age_at_record']

# Rolling statistics
data['sdi_rolling_mean'] = data.groupby('PTNO')['score_n'].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean()
)
data['flares_rolling_sum'] = data.groupby('PTNO')['total_flares'].transform(
    lambda x: x.rolling(window=3, min_periods=1).sum()
)

# Target encoding with rare class handling
class_counts = data['end_state'].value_counts()
rare_classes = class_counts[class_counts < 10].index
data['end_state'] = data['end_state'].replace(rare_classes, 'Other')
label_encoder = LabelEncoder()
data['end_state_encoded'] = label_encoder.fit_transform(data['end_state'])

# --- Automatic Feature Selection from Numeric Variables ---
# Select all numeric columns excluding dates and identifiers
numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns
numeric_cols = [col for col in numeric_cols if col not in ['PTNO', 'end_state_encoded']]

# Add engineered features to the numeric columns list
engineered_features = [
    'time_since_first', 'time_since_last',
    'steroid_flare_interaction', 'sdi_age_interaction',
    'sdi_rolling_mean', 'flares_rolling_sum'
]
numeric_cols = list(set(numeric_cols).union(set(engineered_features)))

# --- Handle Missing and Infinite Values ---
# Fill NA with 0 for all numeric columns
data[numeric_cols] = data[numeric_cols].fillna(0)

# Replace infinite values with large finite numbers
data[numeric_cols] = data[numeric_cols].replace([np.inf, -np.inf], np.nan)
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].max())

# Feature scaling
scaler = MinMaxScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

# --- Patient-level Aggregation ---
agg_dict = {
    **{col: 'last' for col in numeric_cols},  # Most recent observation
    'total_flares': 'sum',                   # Total flares over time
    'severe_flare': 'sum',                   # Total severe flares
    'score_n': 'max',                        # Worst SDI score
    'SLEDAI_Constitutional': 'max',          # Worst constitutional symptoms
    'STERDOSE': 'mean'                       # Average steroid dose
}

X = data.groupby('PTNO').agg(agg_dict)
y = data.groupby('PTNO')['end_state_encoded'].last()

# --- Hyperparameter Tuning Setup ---
# Define the pipeline
pipeline = Pipeline([
    ('smote', SMOTE(sampling_strategy='auto', random_state=42)),
    ('feature_selection', SelectFromModel(
        BalancedRandomForestClassifier(n_estimators=100, random_state=42))
    ),
    ('classifier', BalancedRandomForestClassifier(random_state=42))
])

# Define parameter distributions for randomized search
param_dist = {
    'feature_selection__max_features': [0.5, 0.7, 0.9, None],
    'feature_selection__threshold': ['median', 'mean', 0.01, 0.05],
    'classifier__n_estimators': randint(200, 600),
    'classifier__max_depth': randint(5, 20),
    'classifier__min_samples_split': randint(2, 10),
    'classifier__min_samples_leaf': randint(1, 5),
    'classifier__max_features': ['sqrt', 'log2', 0.5, 0.7],
    'classifier__class_weight': ['balanced', 'balanced_subsample']
}

# Scoring metric
scorer = make_scorer(balanced_accuracy_score)

# --- Cross-Validated Hyperparameter Tuning ---
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=50,  # Number of parameter settings sampled
    cv=skf,
    scoring=scorer,
    n_jobs=-1,
    random_state=42,
    verbose=2
)

search.fit(X, y)

# --- Best Model Evaluation ---
print("\n=== Best Parameters ===")
print(search.best_params_)

best_model = search.best_estimator_

# Feature importance from best model
feature_selector = best_model.named_steps['feature_selection']
selected_features = feature_selector.get_support()
selected_feature_names = X.columns[selected_features]

print("\n=== Selected Features ===")
print(selected_feature_names)

# Get feature importances from the final classifier
importances = best_model.named_steps['classifier'].feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': selected_feature_names,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print("\n=== Feature Importances ===")
print(feature_importance_df.head(20))

# --- Cross-Validation with Best Model ---
cv_results = {
    'balanced_accuracy': [],
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': []
}

for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    cv_results['balanced_accuracy'].append(balanced_accuracy_score(y_test, y_pred))
    cv_results['accuracy'].append(accuracy_score(y_test, y_pred))
    cv_results['precision'].append(precision_score(y_test, y_pred, average='weighted'))
    cv_results['recall'].append(recall_score(y_test, y_pred, average='weighted'))
    cv_results['f1'].append(f1_score(y_test, y_pred, average='weighted'))

    print(f"\nFold {fold + 1} Classification Report:")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# --- Final Results ---
print("\n=== Final Performance ===")
print(f"Mean Balanced Accuracy: {np.mean(cv_results['balanced_accuracy']):.4f} (±{np.std(cv_results['balanced_accuracy']):.4f})")
print(f"Mean Accuracy: {np.mean(cv_results['accuracy']):.4f}")
print(f"Mean Precision: {np.mean(cv_results['precision']):.4f}")
print(f"Mean Recall: {np.mean(cv_results['recall']):.4f}")
print(f"Mean F1: {np.mean(cv_results['f1']):.4f}")

# Save the best model
import joblib
joblib.dump(best_model, 'best_sle_model.pkl')

Fitting 5 folds for each of 50 candidates, totalling 250 fits

=== Best Parameters ===
{'classifier__class_weight': 'balanced', 'classifier__max_depth': 6, 'classifier__max_features': 0.5, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 6, 'classifier__n_estimators': 312, 'feature_selection__max_features': None, 'feature_selection__threshold': 'median'}

=== Selected Features ===
Index(['SLED11_I', 'time_first_last', 'AGE_DX', 'cum_time', 'SDI_Ocular',
       'AVASCNEC', 'SLEDAI_Hematological', 'STERAVER', 'SLED17_I', 'SLED15_I',
       'SEIZURX6', 'sick_leave_spell', 'mild_flare', 'SLED13_I',
       'SLEDAI_Other', 'SLED24_I', 'AMDOSE', 'SDI_Dermatologic', 'PROT24',
       'ACATARACT', 'mild_flares', 'SLEDAI2_I', 'SLED21_I',
       'sick_leave_duration', 'SDI_Musculoskeletal', 'total_visits',
       'SLEDAI_Neurological', 'ISAVER', 'ISDOSE', 'SDI_Neuropsychiatric',
       'ISMETH', 'SLED12_I', 'STERDUR', 'EMP_numeric', 'high_dose',
       'SDI_Endocrine', 'sdi_age_

['best_sle_model.pkl']